In [83]:
# DF 
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from math import floor
from scipy.optimize import minimize

# visualization 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px # type: ignore
import plotly.graph_objs as go # type: ignore

# # time series
import yfinance as yf
from statsmodels.tsa.stattools import adfuller, kpss # Tests for stationarity
from statsmodels.tsa.statespace.sarimax import SARIMAX # Actual Model
from pmdarima.arima import auto_arima
import pmdarima
from statsmodels.tsa.seasonal import seasonal_decompose # Decomposing into trend, seasonal, residual
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf # Plotting (Partial) Autocorrelation
from statsmodels.tsa.stattools import acf, pacf # Calculating (Partial) Autocorrelation Functions
from scipy.stats import normaltest, boxcox # Tests if data came from a normal distribution, is a transformation

# Evaluation
from sklearn.metrics import mean_squared_error # A metric for evaluating a model
from sklearn.model_selection import train_test_split

# To ensure reproducability
import random

# To see progress
from tqdm import tqdm

# pipelines and regressions
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score


### Loading the data

In [62]:
stocks = pd.read_csv('sp500.csv')
stocks.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [63]:
idx = 269
ticker, company = stocks['Symbol'].iloc[idx], stocks['Security'].iloc[idx]
print(ticker, company)

JPM JPMorgan Chase


In [64]:
start_date = '2021-01-01'
end_date = date.today()

In [65]:
df = yf.download(ticker, start = start_date, end = end_date)
df.to_csv('data.csv')
df = pd.read_csv('data.csv')
df

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,127.500000,127.860001,124.779999,125.870003,113.580292,16819900
1,2021-01-05,124.989998,126.300003,123.769997,125.650002,114.198334,13731200
2,2021-01-06,129.880005,132.770004,127.879997,131.550003,119.560608,24909100
3,2021-01-07,135.690002,138.190002,134.919998,135.869995,123.486877,21940400
4,2021-01-08,135.970001,136.350006,134.119995,136.020004,123.623222,12035100
...,...,...,...,...,...,...,...
881,2024-07-08,205.039993,206.899994,203.970001,205.169998,205.169998,8707000
882,2024-07-09,205.630005,209.759995,205.449997,207.630005,207.630005,9058900
883,2024-07-10,206.139999,207.970001,205.580002,207.800003,207.800003,8328500
884,2024-07-11,206.210007,208.100006,205.380005,207.449997,207.449997,10658100


In [66]:
df['Daily Returns'] = df['Close'].pct_change().fillna(0)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Returns
0,2021-01-04,127.500000,127.860001,124.779999,125.870003,113.580292,16819900,0.000000
1,2021-01-05,124.989998,126.300003,123.769997,125.650002,114.198334,13731200,-0.001748
2,2021-01-06,129.880005,132.770004,127.879997,131.550003,119.560608,24909100,0.046956
3,2021-01-07,135.690002,138.190002,134.919998,135.869995,123.486877,21940400,0.032839
4,2021-01-08,135.970001,136.350006,134.119995,136.020004,123.623222,12035100,0.001104


In [67]:
fig1 = px.histogram(df, x='Daily Returns', text_auto=True).update_layout(title=f"Daily Returns for {company}", yaxis_title='Frequency')
fig1.show()

### Feature Engineering

Important features in making a predictive model for financial data:
1. RSI: Relative Strength Index (measures the speed at which prices change; determines if its overbought or oversold)
2. MACD: Moving Average Convergence Distance (momentum oscillator to help identify potential buying/selling opportunities)

In [45]:
# calculating 14 day rsi
delta = df['Close'].diff()
gain= delta.copy()
loss = delta.copy()
gain[gain<0] = 0
loss[loss > 0] = 0
avg_gain = gain.rolling(window=14).mean()
avg_loss = np.abs(loss.rolling(window=14).mean())
rs = avg_gain/avg_loss
rsi = 100 - (100/(1+rs))

In [46]:
# MACD  
ema_12 = df['Close'].ewm(span=12, adjust=False).mean() #ema = estimated moving average, ewm = exponentially weighted moving average
ema_26 = df['Close'].ewm(span=26, adjust=False).mean()
macd = ema_26-ema_12

In [68]:
df['RSI'] = rsi
df['MACD']= macd
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Returns,RSI,MACD
881,2024-07-08,205.039993,206.899994,203.970001,205.169998,205.169998,8707000,0.001856,71.006983,-2.425307
882,2024-07-09,205.630005,209.759995,205.449997,207.630005,207.630005,9058900,0.011990,72.294661,-2.627064
883,2024-07-10,206.139999,207.970001,205.580002,207.800003,207.800003,8328500,0.000819,70.361970,-2.768759
884,2024-07-11,206.210007,208.100006,205.380005,207.449997,207.449997,10658100,-0.001684,67.420603,-2.820300
885,2024-07-12,204.000000,207.449997,202.100006,204.940002,204.940002,15439700,-0.012099,67.048116,-2.628314


### Model Training

In [48]:
X = df[['RSI', 'MACD']]
y = np.where(df['Close'].shift(-1) > df['Close'], 1, -1)

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train in histogram gradient boosting classifier model
model = HistGradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)

pred = model.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print(f"Model Accuracy: {accuracy}")

Model Accuracy: 0.5


### Model Evaluation

In [49]:
df['Predicted Signal'] = model.predict(X)
df['Strategy Returns'] = df['Daily Returns']*df['Predicted Signal'].shift()
df['Cumulative Returns'] = (1+df['Strategy Returns']).cumprod()

In [50]:
fig2 = px.line(df, x='Date', y='Cumulative Returns').update_layout(title = f'Cumulative Returns of the Trading Strategy for {company}')
fig2

## Tests for stationarity

In [51]:
df1 = df[['Date', 'Adj Close']]
df1.head()

,Date,Adj Close
0,2021-01-04,113.580292
1,2021-01-05,114.198334
2,2021-01-06,119.560608
3,2021-01-07,123.486877
4,2021-01-08,123.623222


In [52]:
train_split = floor(0.8*df['Adj Close'].size)
y_train, y_test = df1['Adj Close'].iloc[:train_split], df1['Adj Close'].iloc[train_split:]

### Visualization

In [53]:
# line plot
fig3 = px.line(df1, x='Date', y='Adj Close')
fig3.update_layout(title = f'Stock Price for {company}')
fig3

In [54]:
# The Augmented Dickey-Fuller Test
# Statistical hypothesis test that helps assess stationarity. Null hypothesis: Time series is not stationary

def adf_results(time_series):
    # Use the statsmodels implementation of ADF test
    result = adfuller(time_series)

    # Print results
    print('ADF Statistic: {}\np-value: {}\nCritical Values: {}'.format(result[0], result[1], result[4]))

    # Based on p-value, can determine if stationary
    stationary = (result[1] <= 0.05)
    
    # Conditionally adds the word not
    print('Result is stationary' if stationary else 'Result is NOT stationary')

In [55]:
adf_results(y_train)


ADF Statistic: -2.224487102428002
p-value: 0.19747838032581155
Critical Values: {'1%': -3.439646367660705, '5%': -2.8656425177031375, '10%': -2.5689546724554404}
Result is NOT stationary


In [56]:
kpss(y_train)


(0.695058316150772,
 0.013994698531748003,
 17,
 {'10%': 0.347, '5%': 0.463, '2.5%': 0.574, '1%': 0.739})

In [57]:
diffs = y_train.diff().dropna()
# # Line plot
fig4 = px.line(diffs)
fig4

# Title
fig4.update_layout(title=f"Change in Stock Price for {company}")

# Display the plot
fig4.show()

# Portfolio and Risk Management Test

In [70]:
tickers = ['JPM', 'AAPL', 'MSFT', 'GOOG']
dataset = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

[*********************100%%**********************]  4 of 4 completed


Ticker,AAPL,GOOG,JPM,MSFT
Date,,,,
2021-01-04,126.830070,86.313782,113.580292,211.224319
2021-01-05,128.398178,86.947060,114.198334,211.428101
2021-01-06,124.076088,86.665878,119.560608,205.945892
2021-01-07,128.309982,89.260925,123.486877,211.806503
2021-01-08,129.417435,90.257790,123.623222,213.096970
...,...,...,...,...
2024-07-08,227.820007,190.479996,205.169998,466.239990
2024-07-09,228.679993,190.440002,207.630005,459.540009
2024-07-10,232.979996,192.660004,207.800003,466.250000


In [90]:
total_days = (dataset.index[-1] - dataset.index[0]).days
total_years = total_days / 365.25
avg_days_open = len(dataset) / total_years
returns = dataset.pct_change().dropna()
expected_returns = returns.mean()*avg_days_open
cov_matrix = returns.cov()*avg_days_open

In [82]:
def portfolio_performance(weights, expected_returns, cov_matrix):
    returns = np.dot(weights, expected_returns)
    std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return returns, std_dev

def neg_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate=0.01):
    returns, std_dev = portfolio_performance(weights, expected_returns, cov_matrix)
    return -(returns - risk_free_rate) / std_dev

def optimize_portfolio(expected_returns, cov_matrix):
    num_assets = len(expected_returns)
    args = (expected_returns, cov_matrix)
    constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
    bounds = tuple((0, 1) for asset in range(num_assets))
    result = minimize(neg_sharpe_ratio, num_assets * [1. / num_assets], args=args,
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result


In [94]:
# optimum results
opt_result = optimize_portfolio(expected_returns, cov_matrix)
opt_weights = opt_result.x
opt_return, opt_std_dev = portfolio_performance(opt_weights, expected_returns, cov_matrix)
opt_sharpe_ratio = (opt_return - 0.01) / opt_std_dev


In [95]:
# Display results
print("Optimal Weights: ", opt_weights)
print("Expected Portfolio Return: ", opt_return)
print("Expected Portfolio Volatility: ", opt_std_dev)
print("Sharpe Ratio: ", opt_sharpe_ratio)

Optimal Weights:  [0.         0.15145054 0.43982614 0.40872332]
Expected Portfolio Return:  0.23015830755227745
Expected Portfolio Volatility:  0.2035659520368404
Sharpe Ratio:  1.0815085005592402


In [103]:

def plot_efficient_frontier(expected_returns, cov_matrix, num_portfolios=10000, risk_free_rate=0.01):
    results = np.zeros((3, num_portfolios))
    weights_record = []
    for i in range(num_portfolios):
        weights = np.random.random(len(expected_returns))
        weights /= np.sum(weights)
        weights_record.append(weights)
        portfolio_return, portfolio_std_dev = portfolio_performance(weights, expected_returns, cov_matrix)
        results[0,i] = portfolio_return
        results[1,i] = portfolio_std_dev
        results[2,i] = (portfolio_return - risk_free_rate) / portfolio_std_dev
    
    results_df = pd.DataFrame({
        'Returns': results[0, :],
        'Volatility': results[1, :],
        'Sharpe Ratio': results[2, :]
    })

    fig = px.scatter(
        results_df,
        x='Volatility',
        y='Returns',
        color='Sharpe Ratio',
        color_continuous_scale='YlGnBu',
        title='Efficient Frontier',
        labels={'Volatility': 'Risk (Standard Deviation)', 'Returns': 'Expected Returns'}
    )
    
    opt_return, opt_std_dev = portfolio_performance(opt_weights, expected_returns, cov_matrix)
    fig.add_scatter(x=[opt_std_dev], y=[opt_return], mode='markers', marker=dict(color='red', size=15), name='Optimal Portfolio')

    fig.update_layout(
        title={
            'text': "Efficient Frontier",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title="Volatility (Risk)",
        yaxis_title="Expected Returns",
        font=dict(
            family="Courier New, monospace",
            size=12,
            color="RebeccaPurple"
        )
    )

    fig.show()

In [104]:
plot_efficient_frontier(expected_returns, cov_matrix)
